In [1]:
#----------------------------------------------EDIT WITH CAUTION----------------------------------------------------

import numpy as np
import time

def rand_fill(TYPES):
    x = 0
    while x == 0:
        random = np.random.rand(TYPES)
        y = 0 - np.log(random)
        x = np.sum(y)
    pop = y / x
    return pop

def replicate(pop, oldpop, TYPES, matrix, mu, r):
    oldpop = np.copy(pop)
    fitness = np.zeros(TYPES)
    
    for i in range(TYPES):
        for j in range(TYPES):
            payoff_ij = matrix[i][j] * pop[j]
            fitness[i] += r * (matrix[i][i] * pop[i]) + (1 - r) * payoff_ij
    
    avefit = np.sum(fitness * pop)
    
    for i in range(TYPES):
        pop[i] = pop[i] * ((1 - mu) * fitness[i] / avefit) + mu / TYPES
    
    return pop, oldpop

def detect_stable(oldpop, pop, threshold=1e-5):
    return np.all(np.abs(oldpop - pop) < threshold)

def print_run(pop, gen, totals):
    print(" ".join(map(str, pop)), " Gen: ", gen)
    totals += pop
    return totals

def print_results(sum_proportions, RUNS):
    mean_proportions = sum_proportions / RUNS
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.0001 else "0.00" for prop in mean_proportions]
    print("\033[1m\nMean Proportions Across Simulations: \033[0m")
    print(" ".join(formatted_proportions))

def main(matrix, mu=0.00, r=0.00, RUNS=100, MAXGEN=100, printruns=False):
    TYPES=matrix.shape[0]
    np.random.seed(int(time.time()))
    
    totals = np.zeros(TYPES)
    sum_proportions = np.zeros(TYPES)
    
    for i in range(RUNS):
        pop = rand_fill(TYPES)
        oldpop = np.zeros(TYPES)
        gen = 0
        stable = False
        
        while not stable and gen < MAXGEN:
            pop, oldpop = replicate(pop, oldpop, TYPES, matrix, mu, r)
            stable = detect_stable(oldpop, pop)
            gen += 1
            
        if printruns:
            totals = print_run(pop, gen, totals)
        sum_proportions += pop
    
    print_results(sum_proportions, RUNS)

## Guide to Running Simulations<br>
### Variables: <br>
<b>Mutation: mu</b><br>
Default value: 0.00<br>
The chance of mutation, from 0 to 1. <br>

<b>Correlation: r</b><br>
Default value: 0.00<br>
The chance of correlation, from 0 to 1. <br>

<b>Number of Runs: runs</b><br>
Default value: 100<br>
The number of times the program runs a randomized simulation according to the provided matrix.<br>
The program will average results from all runs to arrive at a percentage spread of strategies. <br>

<b>Maximum Number of Generations: maxgen</b><br>
Default value: 100<br>
The maximum number of replications in a given simulation.<br>
This number will not be reached by most traditional games.<br>

<b>Print Runs: printruns</b><br>
Default value: False<br>
A boolean (True or False) that determines if the simulation shows results for each individual run in addition to the averaged results.<br>

In [2]:
#----------------------------------------------RUNNING SIMULATIONS----------------------------------------------------
"""
Payoff Matrices
Symmetric games only - payoffs listed are only for player 1
"""
# Prisoner's Dilemma
prisonersdilemma = np.array([[2,0],
                   [3,1]])

# Reduced Ultimatum Game
smallultimatum = np.array([[5,2.5,6,5],
                   [2.5,5,5,0],
                   [4,5,5,1.5],
                   [5,0,3.5,5]])

# Full Ultimatum Game
fullultimatum = np.array([[5,5,3.5,1.5,6,6,2.5,2.5],
                   [5,5,1.5,0,3.5,3.5,0,0],
                   [1.5,3.5,0,0,6,6,2.5,2.5],
                   [3.5,0,0,0,3.5,3.5,0,0],
                   [4,1.5,4,1.5,5,2.5,5,2.5],
                   [4,1.5,4,1.5,2.5,0,5,0],
                   [2.5,0,2.5,0,5,5,5,2.5],
                   [2.5,0,2.5,0,2.5,0,2.5,0]])

# Change a value x by setting it to a value e.g. main(matrix, r=0.5, printruns=True)
main(smallultimatum)


Mean Proportions Across Simulations: 
0.518 0.163 0.067 0.252


Full Ultimatum:

1: Gamesman

2: Mad Dog

3: Dumb

4: Pure Spite

5: Easy Rider

6: Dumber

7: Fairman

8: Control Freak

With no mutation and no correlation, Gamesman tends to take the cake most of the time, with varied spreads of the other three dominant strategies. 

If you increase mutation, there becomes a slightly higher chance of other strategies creeping in.

The higher your correlation, the more consistent the spread becomes - since all four strategies get (5,5) when paired against themselves, correlation dilutes the gamesman's advantage. 